### Import Libraries

In [8]:
import pandas as pd
import numpy as np

import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

### Read Dataset

In [9]:
url= 'https://raw.githubusercontent.com/natashayulian/diamond_dataset/master/diamonds.csv'
df = pd.read_csv(url)

### Data Splitting

In [10]:
train, test = train_test_split(df, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

34521 train examples
8631 validation examples
10788 test examples


In [11]:
#Membuat kolom target
# 0 = low; 1 = high
df['target'] = np.where(df['price'] <= 1000, 0, 1)
# Drop un-used columns.
df = df.drop(columns=['price'])

### Membungkus Dataframe

Bungkus dataframe dengan tf.data agar memungkinkan untuk menggunakan feature column sebagai jembatan dari dataframe menjadi fitur yang digunakan untuk train model. Jika file CSV nya sangat besar, kita harus menggunakan tf.data untuk membaca file tersebut dari harddisk.

In [12]:
# Cara untuk membuat dataset tf.data dari pandas dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = df.copy()
  labels = dataframe.pop('target')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds
batch_size = 10 #bath ukuran kecil untuk demonstrasi
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

### Feature Column

#### Numeric Column
Numeric column merupakan jenis feature column yang paling sederhana dan digunakan untuk merepresentasikan sebuah fitur dengan value yang apa adanya. Masukkan ke TensorFlow harus berupa angka dan numeric column sudah merupakan angka sehingga tidak perlu diubah lagi.

In [20]:
example_batch = next(iter(train_ds))[0]
def demo(feature_column):
  feature_layer = layers.DenseFeatures(feature_column)
  print(feature_layer(example_batch).numpy())

In [22]:
#numeric column
carat = feature_column.numeric_column('carat')
demo(carat)

AttributeError: module 'keras._tf_keras.keras.layers' has no attribute 'DenseFeatures'

#### Bucketized Column
Terkadang kita memiliki dataset numerik yang memiliki value beragam dengan range cukup jauh. Daripada memasukkan setiap value ke dalam numeric column, kita dapat menggunakan bucketized column untuk membagi value-value tersebut.

In [21]:
#bucketized column
carat = feature_column.numeric_column('carat')
carat_buckets = feature_column.bucketized_column(carat, boundaries=[1, 2])
demo(carat_buckets)

AttributeError: module 'keras._tf_keras.keras.layers' has no attribute 'DenseFeatures'

#### Categorical Column
Kita tidak dapat memasukkan value String ke dalam TensorFlow. Oleh karena itu, categorical column digunakan untuk mengubah value String pada dataset (pada dataset ini contohnya cut dan clarity) menjadi angka.

In [23]:
#categorical
color_type = feature_column.categorical_column_with_vocabulary_list(
      'color', ['E', 'I','J','D','H', 'G','F'])

color_type_one_hot = feature_column.indicator_column(color_type)
demo(color_type_one_hot)

Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.
Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.


AttributeError: module 'keras._tf_keras.keras.layers' has no attribute 'DenseFeatures'

#### Embedding Column
Jika kita memiliki banyak value dalam satu categorical column, lalu jenis value tersebut dapat bertambah seiring dengan berjalannya waktu, data tersebut menjadi tidak cocok jika hanya direpresentasikan dengan nilai 0 atau 1 seperti categorical column. Embedding column merepresentasikan satu categorical column dengan nilai yang beragam.

In [ ]:
#embedding
clarity = feature_column.categorical_column_with_vocabulary_list(
      'clarity', df.clarity.unique())
clarity_embedding = feature_column.embedding_column(clarity, dimension=6)
demo(clarity_embedding)

#### Hashed Feature Column
Hashed feature column merupakan cara alternatif untuk merepresentasikan categorical column yang memiliki banyak jenis value. Kita dapat menentukan jumlah hash_buckets jauh lebih sedikit dari jumlah kategori yang sebenarnya untuk menghemat tempat.

In [ ]:
#hashed feature
clarity_hashed = feature_column.categorical_column_with_hash_bucket(
      'clarity', hash_bucket_size=5)
demo(feature_column.indicator_column(clarity_hashed))

#### Crossed Feature Column
Crossed feature column menggabungkan banyak feature column menjadi satu feature column. Crossed feature column membuat column feature baru yang memungkinkan model untuk mempelajari weight berbeda untuk setiap kombinasi dari column feature.

In [ ]:
#cross feature
#data yang di cross harus berupa string, categorical, atau bucketized
crossed_feature = feature_column.crossed_column([carat_buckets, color_type],
                                                hash_bucket_size=10)
demo(feature_column.indicator_column(crossed_feature))

### Membuat Feature Layer

In [ ]:
#Pilih feature column mana yang akan digunakan
feature_columns = []
# numeric column
for header in ['carat', 'depth', 'x', 'y', 'z']:
  feature_columns.append(feature_column.numeric_column(header))

#membuat feature layer
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)
batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

### Train Model

In [ ]:
#create, compile, and train the model
model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(128, activation='relu'),
  layers.Dense(128, activation='relu'),
  layers.Dropout(.1),
  layers.Dense(1)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(train_ds,
          validation_data=val_ds,
          epochs=10)